In [ ]:
import PIL
from PIL import Image
import numpy as np

In [ ]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import os
os.environ["PROJ_LIB"] = "C:\\Utilities\\Python\\Anaconda\\Library\\share"; #fixr
from mpl_toolkits.basemap import Basemap

## Creating 2D Images of all layers before putting it into GIF

In [ ]:
air_file = nc.Dataset('data/air.mon.meanv3.nc')
air_lons = air_file.variables['lon'][:]
air_lats = air_file.variables['lat'][:]
pressure = air_file.variables['level'][:]
air_size = air_file.variables['air'][0].size
layer = air_file.variables['air'][0,0].size
xxair, yyair = np.meshgrid(air_lons, air_lats)

water_file = nc.Dataset('data/pottmp.1998.nc')
water_lons = water_file.variables['lon'][:]
water_lats = water_file.variables['lat'][:]
xxwater, yywater = np.meshgrid(water_lons, water_lats)

In [ ]:
import matplotlib.colors as colors
colors1 = plt.cm.winter(np.linspace(0, 1, 500))
colors2 = plt.cm.hot_r(np.linspace(0, 0.9, 500))

# combine them and build a new colormap
color = np.vstack((colors1, colors2))
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', color)

In [ ]:
#For this example, we will look at January. Works for all months
eofs = np.load('January Physical EOFs.npy')

#Making Water 2D Image
ig = plt.figure(figsize=(20,8))
m = Basemap(projection = 'cyl', llcrnrlon = 0, urcrnrlon = 360,
            llcrnrlat = -90, urcrnrlat = 90, resolution = 'c')
#0 for mode 1. For different mode, change 0 to mode-1
Z = 0-np.reshape(eofs[air_size:,0], xxwater.shape)
zmin = np.nanmin(Z)
zmax = np.nanmax(Z)
zlim = np.minimum(np.abs(zmin), zmax)
surface = np.clip(Z, -zlim, zlim)
vmax = np.nanmax(surface)
vmin = np.nanmin(surface)

norm = colors.TwoSlopeNorm(vmin = vmin, vcenter=0.0, vmax = vmax)
m.pcolormesh(xxwater,yywater,surface,norm = norm, shading='auto',cmap=mymap)
cb = m.colorbar(location='right')
cb.ax.tick_params(labelsize=20)
m.drawcoastlines()
m.drawmapboundary()
m.drawparallels(np.arange(-90.,90.,30.),labels=[1,0,0,0], size = 20)
m.drawmeridians(np.arange(-180.,180.,60.),labels=[0,0,0,1], size = 20)
plt.title('January Physical EOFs at  5m. Mode 1' , size = 25)

plt.savefig('images/January Air and Ocean Physical EOF Mode 1 1')


In [ ]:
for i in range(0,pressure.size-1):
    ig=plt.figure(figsize=(20, 8) )


    m=Basemap(projection = 'cyl', llcrnrlon = 0, urcrnrlon = 360,
            llcrnrlat = -90, urcrnrlat = 90, resolution = 'c')

    Z =0-np.reshape(eofs[layer*i:layer*(i+1),0],xxair.shape)
    zmin = np.nanmin(Z)
    zmax = np.nanmax(Z)
    zlim = np.minimum(np.abs(zmin), zmax)
    surface = np.clip(Z, -zlim, zlim)        
    vmax = np.nanmax(surface)
    vmin = np.nanmin(surface)
    norm = colors.TwoSlopeNorm(vmin = vmin, vcenter=0.0, vmax = vmax)

    m.pcolormesh(xxair,yyair,surface,norm = norm, shading='auto',cmap=mymap)
    cb = m.colorbar(location='right')
    cb.ax.tick_params(labelsize=20)
    m.drawcoastlines()
    m.drawmapboundary()
    m.drawparallels(np.arange(-90.,90.,30.),labels=[1,0,0,0], size = 20)
    m.drawmeridians(np.arange(-180.,180.,60.),labels=[0,0,0,1], size = 20)
    plt.title('January Air Physical EOFs at '+ str(pressure[i]) + ' mbar. Mode 1', size = 25)
    plt.savefig('images/January Air and Ocean Physical EOF Mode 1 '+str(i+2) + '.png' )

## Compiling Images to make a GIF

In [ ]:
os.chdir('images')

In [42]:
imageFrames = []
r = np.arange(1,pressure.size)
for k in r:
    newFrame = PIL.Image.open(r'January Air and Ocean Physical EOF Mode 1 '+str(k)+'.png')
    imageFrames.append(newFrame)

imageFrames[0].save('Ocean and Air Lapse January EOFs Mode 1', format = 'GIF', 
                   append_images = imageFrames[1:], duration = 500,
                   save_all = True, loop = 0)